In [1]:
import pybaseball as pyb
import pandas as pd
import numpy as np

Read in the data.

In [2]:
df = pd.read_csv('raw_mvp_prediction_data.csv')

/var/folders/pz/pxdv_rx91j70b52l206jy3qr0000gn/T/ipykernel_13825/3098505661.py:1: DtypeWarning: Columns (58,375) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_mvp_prediction_data.csv')
